**BRD 2 26/8/25**

In [ ]:
import requests
import json
import re

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test (small, code-focused, quantized for 16GB RAM)
models = [                  # ~1GB, code-focused     "deepseek-coder:1.3b"      # ~2GB, code-focused        "deepseek-coder:6.7b",
    "codellama:7b-instruct",     # ~4GB, code-focused
    "qwen2.5-coder:7b"     # ~300MB, general-purpose (backup)
]

code_instruction = """
You are a Frontend Developer. Output a single VALID JSON object. 
Top-level key MUST be "frontend". Each key is a file path, each value is the full file content as a string.
Absolutely NO markdown, NO code fences, NO comments, NO prose. Just JSON.

Use ONLY: React, Redux (plain or @reduxjs/toolkit), Tailwind CSS, Vite, PapaParse, uuid, react-router-dom.
Do NOT include axios, react-toastify, or any other libraries.

Rules:
- Mandatory files: 
  src/App.jsx, src/main.jsx, src/index.css, src/store/store.js, src/AppRouter.jsx, tailwind.config.js, vite.config.js, package.json
- Include reducers AND actions for the main entity (tasks).
  Reducers/Actions MUST implement: add, edit, delete, toggle complete, assign (with role support).
- Components REQUIRED (fully implemented with JSX + Redux hooks): 
  src/components/TaskList.jsx, src/components/TaskItem.jsx, src/components/TaskForm.jsx, 
  src/components/ManagerDashboard.jsx, src/components/LoginComponent.jsx, 
  src/components/NotificationSystem.jsx, src/components/CSVExport.jsx
- State persistence MUST use localStorage (load on init, save on changes).
- React Router v6 syntax.
- package.json MUST only include the allowed deps above.
- Use double quotes in JSON. Escape all newlines inside strings as \n. 
- No trailing commas anywhere in JSON or code strings.

"""

# Your selected arch_text from Agent 2
arch_text = """
**System Architecture Specification (SAS) for Frontend-only React Application**

## Tech Stack

1. **Frontend Framework**: React.js is the primary technology used to build this application, providing a dynamic and interactive user interface. 
2. **Styling**: Tailwind CSS is employed for utility-first styling approach which allows for rapid customization and scalability.
3. **Build Tools**: Vite is utilized for fast and efficient builds of the React app. It offers features like hot module replacement (HMR) that speeds up development time.
4. **Libraries**: Redux, a predictable state container used to manage the global states of the application, as well as other libraries such as react-redux and redux-thunk for better management of side effects in React applications. LocalStorage is used client-side storage mechanism for persisting data like project data.

## Components

1. **Home Component**: A hero section with a background image/video, heading, and brief summary.
2. **About Component**: A bio section with a profile picture, heading, and short description.
3. **Projects Component**: Project cards are displayed using the Project component, paginated for showing multiple projects. Each project card includes an image, title, short description, and buttons to view more details or visit external links (e.g., GitHub, Live Demo).
4. **Skills Component**: A list of skills with icons and brief descriptions.
5. **Contact Component**: A simple form with fields for name, email, and message. 

## Data Storage

1. **Client-side storage mechanism**: LocalStorage is used to store project data in the browser's memory. The `JSON.stringify()` method is used to serialize (convert into string format) the JavaScript object before storing it in localStorage, and `JSON.parse()` for deserializing back into a JavaScript object when needed.

## UI Features 

1. **Responsive Design**: The application follows a responsive design approach ensuring optimal user experience across various devices including desktops, tablets, and mobile phones.
2. **Notifications**: Alerts or modals are used for notifications like errors or successful form submissions. These can be simulated using the browser's built-in alert function or other UI libraries in React.
3. **Reports Generation**: This is not explicitly mentioned but it could include generating a report based on the user interactions with the website e.g., page views, form submissions etc. 
4. **Access Control**: No specific access control features are required for this single-user portfolio website as there's no login functionality in place.
"""

prompt = code_instruction + arch_text

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
        "model": model,
        "prompt": prompt,
        "stream": False,
        # Many models honor this; some ignore, but it helps:
        "format": "json",
        "options": {
        "num_ctx": 8192,
        "temperature": 0.2,
        # Stop if the model starts writing fences or prose:
        "stop": ["```", "\n```", "\nJSON", "\njson"]
    }
})
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"
    
def clean_json_output(output: str) -> str:
    # 1) Strip common markdown/code fences
    output = re.sub(r"^```(?:json)?\s*|\s*```$", "", output.strip(), flags=re.IGNORECASE|re.MULTILINE)

    # 2) Keep only the largest outermost JSON object
    m = re.search(r"\{[\s\S]*\}", output)
    if m:
        output = m.group(0)
    else:
        return output.strip()  # will fail later -> you'll see the parse error

    # 3) Remove backticks (template literals)
    output = output.replace("`", "")

    # 4) Remove // and /* */ comments inside json-like strings (best-effort)
    output = re.sub(r"//.*?$", "", output, flags=re.MULTILINE)
    output = re.sub(r"/\*[\s\S]*?\*/", "", output)

    # 5) Ensure newlines inside JSON string values are escaped
    #    (only a heuristic; your JSON should already be valid)
    output = re.sub(r'(:\s*")((?:\\.|[^"\\])*)\n', lambda m: m.group(0).replace("\n", r"\n"), output)

    # 6) Remove trailing commas before } or ]
    output = re.sub(r",\s*([}\]])", r"\1", output)

    return output.strip()
#So we end up with valid JSON

def strip_disallowed_deps(frontend: dict) -> None:
    allowed = {
        "react", "react-dom", "redux", "react-redux", "react-router-dom",
        "tailwindcss", "vite", "@vitejs/plugin-react", "papaparse", "uuid",
        "@reduxjs/toolkit"  # if you allow RTK
    }
    pkg = frontend.get("package.json")
    if not pkg:
        return
    try:
        pkg_obj = json.loads(pkg)
        for sec in ["dependencies", "devDependencies"]:
            if sec in pkg_obj:
                pkg_obj[sec] = {k: v for k, v in pkg_obj[sec].items() if k in allowed}
        frontend["package.json"] = json.dumps(pkg_obj, separators=(",", ":"), ensure_ascii=False)
    except Exception:
        pass



def extract_components_and_features(arch_text):
    """
    Extract components, features, and primary entity from arch_text for dynamic scoring.
    Returns components (list), features (list), entity (str).
    """
    components = []
    features = []
    
    # Extract components
    component_section = re.search(r'## Components\b(.*?)##', arch_text, re.DOTALL)
    if component_section:
        component_text = component_section.group(1)
        components = re.findall(r'- (\w+)', component_text)
    
    # Add implied form component for any list component with create/edit
    for comp in components:
        if "List" in comp and comp.replace("List", "Form") not in components:
            components.append(comp.replace("List", "Form"))
    
    # Identify primary data entity (first component ending in 'List')
    entity = next((comp.replace("List", "").lower() for comp in components if comp.endswith("List")), "data")
    
    # Extract features
    feature_terms = ["localstorage", "papaparse", "uuid", "react-router-dom", 
                     "role-based access", "csv report", "notifications", 
                     "error handling", "code splitting", "lazy loading", 
                     "wcag", "responsive design"]
    features.extend(term for term in feature_terms if term.lower() in arch_text.lower())
    
    return components, features, entity


def score_code_output(output, arch_text):
    """
    Score the code output dynamically based on arch_text:
    1. Section presence (frontend)
    2. JSON validity
    3. Line count (100-500)
    4. Component and feature coverage
    """
    # 1️⃣ Clean and parse JSON
    try:
        clean_output = clean_json_output(output)
        output_json = json.loads(clean_output)
        section_score = 1 if "frontend" in output_json and isinstance(output_json["frontend"], dict) and len(output_json["frontend"]) > 0 else 0
    except Exception as e:
        print(f"JSON parse error: {e}")
        section_score = 0
        output_json = {}
    
    # 2️⃣ JSON validity
    json_score = 1 if section_score else 0
    
    # 3️⃣ Line count
    try:
        total_lines = sum(
            len(str(code).splitlines())
            for code in output_json.get("frontend", {}).values()
            if str(code).strip() and not str(code).startswith("<<") and not str(code).startswith("path/to")
        )
        length_score = 1 if 100 <= total_lines <= 1000 else 0
    except Exception as e:
        print(f"Line count error: {e}")
        total_lines = 0
        length_score = 0
    
    # 4️⃣ Component and feature coverage
    components, features, entity = extract_components_and_features(arch_text)
    mandatory_files = [
        "src/App.jsx", "src/main.jsx", "src/index.css",
        "src/store/store.js",
        f"src/store/reducers/{entity}Reducer.js",
        f"src/store/actions/{entity}Actions.js",
        "src/AppRouter.jsx",
        "tailwind.config.js", "vite.config.js", "package.json"
    ]
    component_files = [f"src/components/{comp}.jsx" for comp in components]
    all_files = mandatory_files + component_files
    
    file_score = sum(1 for f in all_files if f in output_json.get("frontend", {})) / len(all_files) if all_files else 0
    feature_score = sum(1 for feat in features if feat.lower() in output.lower()) / len(features) if features else 0
    accuracy_score = (file_score + feature_score) / 2  # Combine file and feature coverage
    
    # Debug info
    print(f"Debug: Expected components: {components}")
    print(f"Debug: Expected features: {features}")
    print(f"Debug: Primary entity: {entity}")
    print(f"Debug: Expected files: {all_files}")
    print(f"Debug: Matched files: {[f for f in all_files if f in output_json.get('frontend', {})]}")
    print(f"Debug: Matched features: {[feat for feat in features if feat.lower() in output.lower()]}")
    print(f"Debug: Total lines: {total_lines}")
    
    # Total score
    total_score = section_score + json_score + length_score + accuracy_score * 4  # max=1+1+1+4=7
    return total_score, section_score, total_lines, accuracy_score


# Run models and store results
#Loop over each model → generate code → clean → score → store results.
results = []
for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, lines, acc_score = score_code_output(output, arch_text)
    print(f"✅ Total Score: {total:.2f}/7 | Sections: {sec_score}/1 | Lines: {lines} | Accuracy: {acc_score*100:.0f}%\n")
    results.append((model, total))


# Finally ranks models by score and picks the best one.
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]
print("="*60)
print(f"🏆 Best model for Agent 3: {best_model}")

# Re-query best model for clean final output
best_output = query_model(best_model, prompt)
clean_output = clean_json_output(best_output)
obj = json.loads(clean_output)

if "frontend" in obj:
    strip_disallowed_deps(obj["frontend"])

🚀 Model: codellama:7b-instruct
------------------------------------------------------------
{
"frontend": {
"src/App.jsx": "// React component that renders the AppRouter\nconst App = () => {\n  return <AppRouter />;\n};\nexport default App;",
"src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom';\nimport App from './App';\n\nReactDOM.render(<App />, document.getElementById('root'));",
"src/index.css": ".container {\n  margin: 0 auto;\n  padding: 2rem;\n}\n\nbody {\n  font-family: sans-serif;\n}",
"src/store/store.js": "import { configureStore } from '@reduxjs/toolkit';\nimport tasksReducer from './tasksSlice';\n\nexport const store = configureStore({\n  reducer: {\n    tasks: tasksReducer,\n  },\n});",
"src/AppRouter.jsx": "// React Router v6 syntax\nimport { BrowserRouter, Route, Routes } from 'react-router-dom';\nimport TaskList from './components/TaskList';\nimport TaskForm from './components/TaskForm';\nimport ManagerDashboard from './components/ManagerDashb

**BRD 2 Updated**

In [ ]:
import requests
import json
import re

# Ollama API base
OLLAMA_URL = "http://localhost:11434/api/generate"

# Models to test (small, code-focused, quantized for 16GB RAM)
models = [                  
    "codellama:7b-instruct",
    "qwen2.5-coder:7b"     
]

# code_instruction = """
# You are a Frontend Developer. Output a single VALID JSON object.
# Top-level key MUST be "frontend". Each key is a file path, each value is the full file content as a string.
# Absolutely NO markdown, NO code fences, NO comments, NO prose. Just JSON.

# Use ONLY: React, Redux Toolkit (if state management needed), Tailwind CSS, Vite, PapaParse, uuid.
# Always use React + Tailwind for UI. No other frontend frameworks allowed.
# Do NOT include axios, react-toastify, or any other libraries.

# Rules:
# - Mandatory files (must always be present): 
#   src/App.jsx, src/main.jsx, src/index.css, tailwind.config.js, vite.config.js, package.json
# - Also mandatory if PRD/Architecture specifies them (NOT optional): 
#   src/store/store.js (if state management is mentioned),
#   src/screens/* (for each screen listed in architecture),
#   src/components/* (for each component listed in architecture)
# - The structure of components, screens, and Redux slices MUST be based on the entities, features, and navigation described in the PRD/arch_text.
# - If state persistence is needed, use localStorage (load on init, save on changes).
# - For navigation:
#   - If PRD says "single-page scroll", use section IDs + anchor links in Navbar (no react-router-dom).
#   - If PRD says "multi-page routing", then use react-router-dom v6.
# - All components must be implemented as React functional components with JSX.
# - Every file value MUST contain actual valid source code (imports, functions, JSX, etc.). 
#   Do NOT output summaries, placeholders, or meta descriptions.
# - package.json MUST only include the allowed deps above.
# - Use double quotes in JSON. Escape all newlines inside strings as \\n. 
# - No trailing commas anywhere in JSON or code strings.

# The output must strictly follow the architecture specifications provided in arch_text.
# """

code_instruction = f"""
You are a Frontend Developer. Output ONLY a single VALID JSON object like: {{"frontend": {{"file/path": "full code string", ...}}}}. Top-level key MUST be "frontend". Each key is a file path, value is FULL file content as string. NO markdown, NO fences, NO comments/prose. Just raw JSON.

Use ONLY: React 18+, Redux Toolkit (if state needed), Tailwind CSS, Vite, PapaParse, uuid. NO react-router-dom (use sections/anchors unless arch specifies multi-page), NO other libs (e.g., no @headlessui, axios).

Rules:
- ALWAYS include: src/App.jsx (wrap screens with Navbar, sections for smooth scroll if single-page), src/main.jsx (use ReactDOM.createRoot), src/index.css (@tailwind directives; add html {{ scroll-behavior: smooth; }} if smooth nav), tailwind.config.js (content: ['./src/**/*.{{js,jsx}}'], theme extend if needed), vite.config.js (with react plugin), package.json (exact deps: {{"dependencies": {{"react": "^18.2.0", "react-dom": "^18.2.0"}}, "devDependencies": {{"tailwindcss": "^3.4.0", "vite": "^5.0.0", "@vitejs/plugin-react": "^4.0.0"}} if Redux: add "@reduxjs/toolkit": "^2.0.0", "react-redux": "^9.0.0"}}, scripts: {{"dev": "vite", "build": "vite build"}}).
- Include arch-specified: src/store/store.js (if state mentioned; configureStore with slices for entities like nav/projects; localStorage persist), src/screens/* and src/components/* (as listed).
- Structure: Base on arch_text entities/features/nav (e.g., if portfolio: screens=Home/About/etc., components=Navbar/ProjectCard; if task app: add TaskList/Form with Redux actions).
- Implement FULL: Functional JSX components, responsive Tailwind (flex min-h-screen), animations (transition-all duration-300 if smooth UX). NO placeholders—hardcode sample data (e.g., projects: [{{"title": "Proj1", "desc": "..."}}], skills: ['React', 'JS']).
- If state persistence needed: localStorage (load on init, save on changes).
- Navigation: Single-page scroll=section IDs + anchor links; multi-page=react-router-dom v6.
- Escape newlines in strings as \\n. Double quotes only. Valid JSX/code in every file.

Follow arch_text EXACTLY: {arch_text}
"""

# Your selected arch_text from Agent 2
arch_text = """
# Frontend Architecture Specification

## Tech Stack
- React (frontend framework)
- Tailwind CSS (styling library)
- Vite (build tool)
- Redux Toolkit (state management)
- localStorage/sessionStorage (client-side storage)

## Mandatory Files
- src/App.jsx
- src/main.jsx
- public/index.html
- src/index.css
- tailwind.config.js
- vite.config.js
- package.json

## Screens
- src/screens/Home.jsx
- src/screens/About.jsx
- src/screens/Projects.jsx
- src/screens/Skills.jsx
- src/screens/Contact.jsx

## Components
- src/components/Navbar.jsx
- src/components/ProjectCard.jsx
- src/components/ContactForm.jsx
- src/components/SkillList.jsx (for Skills screen)

## State Management
Redux store located in `src/store` with following slices:
- navigation (navbar state management)
- projects (project card expansion/collapse)

## Data Storage
None (localStorage/sessionStorage will be used if data persistence is needed)

## UI Features
- Responsive design using Tailwind CSS utilities
- Animations and transitions for smooth user experience
- Smooth scrolling navigation between screens
- Modern, lightweight design
"""


prompt = code_instruction

def query_model(model, prompt):
    """Send prompt to Ollama model and return response"""
    try:
        response = requests.post(OLLAMA_URL, json={
        "model": model,
        "prompt": prompt,
        "stream": False,
        # Many models honor this; some ignore, but it helps:
        "format": "json",
        "options": {
        "num_ctx": 8192,
        "temperature": 0.2,
        # Stop if the model starts writing fences or prose:
        "stop": ["```", "\n```", "\nJSON", "\njson"]
    }
})
        response.raise_for_status()
        return response.json().get("response", "").strip()
    except Exception as e:
        return f"❌ Error with {model}: {str(e)}"
    
def clean_json_output(output: str) -> str:
    # 1) Strip common markdown/code fences
    output = re.sub(r"^```(?:json)?\s*|\s*```$", "", output.strip(), flags=re.IGNORECASE|re.MULTILINE)

    # 2) Keep only the largest outermost JSON object
    m = re.search(r"\{[\s\S]*\}", output)
    if m:
        output = m.group(0)
    else:
        return output.strip()  # will fail later -> you'll see the parse error

    # 3) Remove backticks (template literals)
    output = output.replace("`", "")

    # 4) Remove // and /* */ comments inside json-like strings (best-effort)
    output = re.sub(r"//.*?$", "", output, flags=re.MULTILINE)
    output = re.sub(r"/\*[\s\S]*?\*/", "", output)

    # 5) Ensure newlines inside JSON string values are escaped
    #    (only a heuristic; your JSON should already be valid)
    output = re.sub(r'(:\s*")((?:\\.|[^"\\])*)\n', lambda m: m.group(0).replace("\n", r"\n"), output)

    # 6) Remove trailing commas before } or ]
    output = re.sub(r",\s*([}\]])", r"\1", output)

    return output.strip()
#So we end up with valid JSON

def strip_disallowed_deps(frontend: dict) -> None:
    allowed = {
        "react", "react-dom", "redux", "react-redux", "react-router-dom",
        "tailwindcss", "vite", "@vitejs/plugin-react", "papaparse", "uuid",
        "@reduxjs/toolkit"  # if you allow RTK
    }
    pkg = frontend.get("package.json")
    if not pkg:
        return
    try:
        pkg_obj = json.loads(pkg)
        for sec in ["dependencies", "devDependencies"]:
            if sec in pkg_obj:
                pkg_obj[sec] = {k: v for k, v in pkg_obj[sec].items() if k in allowed}
        frontend["package.json"] = json.dumps(pkg_obj, separators=(",", ":"), ensure_ascii=False)
    except Exception:
        pass



def extract_components_and_features(arch_text):
    """
    Extract components, features, and primary entity from arch_text for dynamic scoring.
    Returns components (list), features (list), entity (str).
    """
    components = []
    features = []
    
    # Extract components
    component_section = re.search(r'## Components\b(.*?)##', arch_text, re.DOTALL)
    if component_section:
        component_text = component_section.group(1)
        components = re.findall(r'-\s+([A-Za-z0-9 ]+?)(?:Component)?\b', component_text)
        components = [c.replace(" ", "") for c in components]
    
    # Add implied form component for any list component with create/edit
    for comp in components:
        if "List" in comp and comp.replace("List", "Form") not in components:
            components.append(comp.replace("List", "Form"))
    
    # Identify primary data entity (first component ending in 'List')
    entity = next((comp.replace("List", "").lower() for comp in components if comp.endswith("List")), "data")

    # --- Base mandatory files ---
    mandatory_files = [
        "src/App.jsx",
        "src/main.jsx",
        "src/index.css",
        "tailwind.config.js",
        "vite.config.js",
        "package.json",
    ]

    if entity:
        if "reducers" in arch_text.lower():
            mandatory_files.append(f"src/store/reducers/{entity}Reducer.js")
        if "actions" in arch_text.lower():
            mandatory_files.append(f"src/store/actions/{entity}Actions.js")

    # --- Conditionally add AppRouter if arch mentions routing ---
    if "router" in arch_text.lower():
        mandatory_files.append("src/AppRouter.jsx")

    # Extract features
    feature_terms = ["localstorage", "papaparse", "uuid", "react-router-dom", 
                     "role-based access", "csv report", "notifications", 
                     "error handling", "code splitting", "lazy loading", 
                     "wcag", "responsive design"]
    features.extend(term for term in feature_terms if term.lower() in arch_text.lower())
    
    return components, features, entity, mandatory_files


def score_code_output(output, arch_text):
    """
    Score the code output dynamically based on arch_text:
    1. Section presence (frontend)
    2. JSON validity
    3. Line count (100-500)
    4. Component and feature coverage
    """
    # 1️⃣ Clean and parse JSON
    try:
        clean_output = clean_json_output(output)
        output_json = json.loads(clean_output)
        section_score = 1 if "frontend" in output_json and isinstance(output_json["frontend"], dict) and len(output_json["frontend"]) > 0 else 0
    except Exception as e:
        print(f"JSON parse error: {e}")
        section_score = 0
        output_json = {}
    
    # 2️⃣ JSON validity
    json_score = 1 if section_score else 0
    
    # 3️⃣ Line count
    try:
        total_lines = sum(
            len(str(code).splitlines())
            for code in output_json.get("frontend", {}).values()
            if str(code).strip() and not str(code).startswith("<<") and not str(code).startswith("path/to")
        )
        length_score = 1 if 50 <= total_lines <= 2000 else 0
    except Exception as e:
        print(f"Line count error: {e}")
        total_lines = 0
        length_score = 0
    
    # 4️⃣ Component and feature coverage
    components, features, entity, mandatory_files = extract_components_and_features(arch_text)
    component_files = [f"src/components/{comp}.jsx" for comp in components]
    all_files = mandatory_files + component_files
    
    file_score = sum(1 for f in all_files if f in output_json.get("frontend", {})) / len(all_files) if all_files else 0
    feature_score = sum(1 for feat in features if feat.lower() in output.lower()) / len(features) if features else 0
    accuracy_score = (file_score + feature_score) / 2  # Combine file and feature coverage
    
    # Debug info
    print(f"Debug: Expected components: {components}")
    print(f"Debug: Expected features: {features}")
    print(f"Debug: Primary entity: {entity}")
    print(f"Debug: Expected files: {all_files}")
    print(f"Debug: Matched files: {[f for f in all_files if f in output_json.get('frontend', {})]}")
    print(f"Debug: Matched features: {[feat for feat in features if feat.lower() in output.lower()]}")
    print(f"Debug: Total lines: {total_lines}")
    
    # Total score
    total_score = section_score + json_score + length_score + accuracy_score * 4  # max=7
    return total_score, section_score, total_lines, accuracy_score


# Run models and store results
#Loop over each model → generate code → clean → score → store results.
results = []
for model in models:
    print("="*60)
    print(f"🚀 Model: {model}")
    print("-"*60)
    output = query_model(model, prompt)
    print(output)
    total, sec_score, lines, acc_score = score_code_output(output, arch_text)
    print(f"✅ Total Score: {total:.2f}/7 | Sections: {sec_score}/1 | Lines: {lines} | Accuracy: {acc_score*100:.0f}%\n")
    results.append((model, total))


# Finally ranks models by score and picks the best one.
results.sort(key=lambda x: x[1], reverse=True)
best_model = results[0][0]
print("="*60)
print(f"🏆 Best model for Agent 3: {best_model}")

# Re-query best model for clean final output
best_output = query_model(best_model, prompt)
clean_output = clean_json_output(best_output)
obj = json.loads(clean_output)

if "frontend" in obj:
    strip_disallowed_deps(obj["frontend"])

🚀 Model: codellama:7b-instruct
------------------------------------------------------------
{
"frontend": {
"src/App.jsx": "import React from 'react';\nimport Navbar from './components/Navbar';\nimport Home from './screens/Home';\nimport About from './screens/About';\nimport Projects from './screens/Projects';\nimport Skills from './screens/Skills';\nimport Contact from './screens/Contact';\n\nfunction App() {\nreturn (\n<div className='App'>\n\t<Navbar />\n\t<Home />\n\t<About />\n\t<Projects />\n\t<Skills />\n\t<Contact />\n</div>);\n}\nexport default App;",
"src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom';\nimport App from './App';\n\nReactDOM.render(<App />, document.getElementById('root'));",
"public/index.html": "<!DOCTYPE html>\n<html lang='en'>\n<head>\n\t<meta charset='UTF-8' />\n\t<meta name='viewport' content='width=device-width, initial-scale=1.0' />\n\t<title>Frontend Architecture</title>\n</head>\n<body>\n\t<div id='root'></div>\n</body>\n</ht

In [ ]:
=== Generated Code (React) ===

{
  "frontend": {
    "src/App.jsx": "import React from 'react';\nimport Navbar from './components/Navbar';\nimport Home from './screens/Home';\nimport About from './screens/About';\nimport Projects from './screens/Projects';\nimport Skills from './screens/Skills';\nimport Contact from './screens/Contact';\n\nfunction App() {\n  return (\n    <div className=\"min-h-screen\">\n      <Navbar />\n      <main className=\"flex flex-col items-center justify-center min-h-screen\">\n        <Route path=\"/\" exact component={Home} />\n        <Route path=\"/about\" component={About} />\n        <Route path=\"/projects\" component={Projects} />\n        <Route path=\"/skills\" component={Skills} />\n        <Route path=\"/contact\" component={Contact} />\n      </main>\n    </div>\n  );\n}\n\nexport default App;",
    "src/main.jsx": "import React from 'react';\nimport ReactDOM from 'react-dom/client';\nimport App from './App';\n\nconst root = ReactDOM.createRoot(document.getElementById('root'));\nroot.render(\n  <React.StrictMode>\n    <App />\n  </React.StrictMode>\n);",
    "src/index.css": "@tailwind base;\n@tailwind components;\n@tailwind utilities;",
    "tailwind.config.js": "module.exports = {\n  content: [\n    './src/**/*.{js,jsx,ts,tsx}',\n  ],\n  theme: {\n    extend: {},\n  },\n  plugins: [],\n};",
    "vite.config.js": "import { defineConfig } from 'vite';\nimport react from '@vitejs/plugin-react';\n\nexport default defineConfig({\n  plugins: [react()],\n});",
    "package.json": "{\n  \"name\": \"personal-portfolio\",\n  \"version\": \"1.0.0\",\n  \"scripts\": {\n    \"start\": \"vite\",\n    \"build\": \"vite build\"\n  },\n  \"dependencies\": {\n    \"react\": \"^18.2.0\",\n    \"react-dom\": \"^18.2.0\",\n    \"tailwindcss\": \"^3.2.4\",\n    \"@vitejs/plugin-react\": \"^2.0.0\"\n  }\n}"
  }
}

**Building agents (1-3)**

In [ ]:
import json
import os

class Memory:
    def __init__(self, storage_file="memory.json"):
        self.storage_file = storage_file
        self.data = {}
        self.load()

    def load(self):
        if os.path.exists(self.storage_file):
            with open(self.storage_file, "r") as f:
                self.data = json.load(f)

    def save(self):
        with open(self.storage_file, "w") as f:
            json.dump(self.data, f, indent=2)

    def set(self, key, value):
        self.data[key] = value
        self.save()

    def get(self, key, default=None):
        return self.data.get(key, default)


In [ ]:
import requests

OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(model, prompt):
    response = requests.post(OLLAMA_URL, json={
        "model": model,
        "prompt": prompt,
        "stream": False
    })
    return response.json().get("response", "").strip()


# --- Agent 1: PRD Generator ---
def run_agent1(memory, brd_text):
    instruction = """You are a Product Manager. Convert BRD into minimal PRD..."""
    prompt = instruction + brd_text
    result = query_ollama("qwen2.5:0.5b-instruct", prompt)
    memory.set("prd", result)
    return result

# --- Agent 2: Architecture Generator ---
def run_agent2(memory):
    prd_text = memory.get("prd")
    instruction = """You are a Solutions Architect. Convert PRD into Architecture Spec..."""
    prompt = instruction + prd_text
    result = query_ollama("codellama:7b-instruct", prompt)
    memory.set("arch", result)
    return result

# --- Agent 3: Code Generator ---
def run_agent3(memory):
    arch_text = memory.get("arch")
    instruction = """You are a Frontend Developer. Output full project JSON code..."""
    prompt = instruction + arch_text
    result = query_ollama("qwen2.5-coder:7b", prompt)
    memory.set("code", result)
    return result


# --- Workflow Runner ---
def workflow(brd_text):
    memory = Memory()

    print("\n🚀 Agent 1 (PRD Generator)...")
    prd = run_agent1(memory, brd_text)
    print(prd)

    print("\n🚀 Agent 2 (Arch Generator)...")
    arch = run_agent2(memory)
    print(arch)

    print("\n🚀 Agent 3 (Code Generator)...")
    code = run_agent3(memory)
    print("✅ Code JSON generated and saved in memory.json")

    return code
